In [244]:
import numpy as np

Activation function

In [245]:
def sigmoid(X):
    return 1 / (1 + np.exp(-X))

calculating output at each node

In [246]:
def calc_Net (input_vector, weight_vector):
    return np.dot(input_vector,weight_vector)


In [247]:
def calc_output (input_vector, weight_vector):
    return sigmoid(calc_Net(input_vector, weight_vector))

Data

In [248]:
X = np.array([7, 4])
target = np.array([0,1])

weights_hidden = [[0.1,0.4],
                  [0.2,0.5],
                  [0.3, 0.7]]
                  
weights_outputs = [[0.3,0.4, 0.5],
                  [0.3,0.4, 0.5]]
weights_outputs = np.array(weights_outputs)
weights_hidden = np.array(weights_hidden)
learning_rate = 5

In [249]:
def HiddenLayerOutput_calc (input_vector,weights_hidden):
    HiddenLayerOutput = []
    for i in range (0,3):
        HiddenLayerOutput.append(calc_output(input_vector,weights_hidden[i]))
    HiddenLayerOutput = np.array(HiddenLayerOutput)
    return HiddenLayerOutput


In [250]:
def OutputLayerOutput_calc (HiddenLayerOutput,weights_outputs):
    OutputLayerOutput = []
    for i in range (0,2):
        OutputLayerOutput.append(calc_output(HiddenLayerOutput,weights_outputs[i]))
    OutputLayerOutput = np.array(OutputLayerOutput)
    return OutputLayerOutput



Walking forward to calculate outputs

In [251]:
def forward (input_vector, weights_hidden, weights_outputs):
    weights_hidden = np.array(weights_hidden)
    weights_outputs = np.array(weights_outputs)       

    
    HiddenLayerOutput = HiddenLayerOutput_calc(input_vector,weights_hidden)

    OutputLayerOutput = OutputLayerOutput_calc(HiddenLayerOutput, weights_outputs)

    return OutputLayerOutput
    

In [252]:
Y = forward (X, weights_hidden, weights_outputs)
print (Y)

[0.76061408 0.76061408]


Calculating Error

In [253]:
def calc_Error (Y, target):
    sum = 0.0
    for i in range (0,target.size):
        sum += (Y[i]-target[i])**2
    return sum

In [254]:
Error = calc_Error (Y, target)
print (Error)

0.63583940228813


Walking backward to update weight

At the output layer

In [255]:
def calc_segma_output (target,Y):
    output_segma = []
    for i in range (0, target.size): 
        output_segma.append((target[i]-Y[i]) * Y[i] * (1 - Y[i]))
    output_segma = np.array(output_segma)
    return output_segma


In [256]:
def calc_delta_w (learning_rate,segma,output,delta ):
    r,c = delta.shape
    for i in range (0,r):
        for j in range (0, c):
            delta[i][j] = learning_rate * segma[i] * output[j]
    return delta




In [257]:
def Updated_weight_fun (old_W,delta,updated):
    updated = old_W + delta
    return updated


At hidden layer

In [258]:
def calc_segma_hidden (output_segma,Updated_weight,prev_output):
    hidden_segma = []
    sum = 0.0
    rows, columns = Updated_weight.shape
    for i in range (0,columns):       
        for j in range (0, rows):           
            sum += output_segma[j]*Updated_weight[j][i]    
        segma = sum * prev_output[i]*(1- prev_output[i])
        hidden_segma.append(segma)
    return hidden_segma

In [271]:
def backward (target,Y,X,weights_hidden,weights_outputs,learning_rate):
    output_segma = calc_segma_output (target,Y)
    prev_output = HiddenLayerOutput_calc(X,weights_hidden)

    delta_w = np.ones(((output_segma.size,prev_output.size)))
    delta_w = calc_delta_w (learning_rate,output_segma,prev_output,delta_w)

    Updated_weight = np.ones(((output_segma.size,prev_output.size)))
    Updated_weight = Updated_weight_fun (weights_outputs,delta_w,Updated_weight)

    hidden_segma = calc_segma_hidden (output_segma,Updated_weight,prev_output)
    hidden_segma = np.array(hidden_segma)

    delta_w_hidden = np.ones(((hidden_segma.size,X.size)))
    delta_w_hidden = calc_delta_w (learning_rate,hidden_segma,X,delta_w_hidden)

    Updated_weight_hidden = np.ones(((hidden_segma.size,X.size)))
    Updated_weight_hidden = Updated_weight_fun(weights_hidden,delta_w_hidden,Updated_weight_hidden)

    Y_new = forward (X,Updated_weight_hidden , Updated_weight)
    return Y_new, Updated_weight_hidden,Updated_weight

Y_new, Wh, Wo = backward (target,Y,X,weights_hidden,weights_outputs,learning_rate)
Error = calc_Error (Y_new, target)
print (Error)
Y2, wh2, wo2 = backward (target,Y,X,Wh,Wo,learning_rate)
Error = calc_Error (Y2, target)
print (Error)


0.1188597883129847
0.009255688783104532


Walking forward again

In [288]:
def backProP (target,Y,X,weights_hidden,weights_outputs,learning_rate,iterations):
    y = forward (X, weights_hidden, weights_outputs)
    Error_1 = calc_Error (Y, target)
    print ("first prediction", y)
    print ("first error : ",Error_1)
    print ("--------------------------------------")

    y_new,Wh_new,Wo_new = backward (target,Y,X,weights_hidden,weights_outputs,learning_rate)
    Error = calc_Error (y_new, target)
    print ("second  prediction", y_new)
    print("error in the second iteration :", Error)
    print ("--------------------------------------")

    for i in range (0,iterations):
        y_new,Wh_new,Wo_new = backward (target,Y,X,Wh_new,Wo_new,learning_rate)
        Error = calc_Error (y_new, target)
        print (i+3,"th  prediction", y_new)
        print("error in the",i+3,"th iteration = ", Error)
        print ("--------------------------------------")

iterations = 4
backProP (target,Y,X,weights_hidden,weights_outputs,learning_rate,iterations)

first prediction [0.76061408 0.76061408]
first error :  0.63583940228813
--------------------------------------
second  prediction [0.3146936  0.85918904]
error in the second iteration : 0.1188597883129847
--------------------------------------
3 th  prediction [0.05563125 0.9215089 ]
error in the 3 th iteration =  0.009255688783104532
--------------------------------------
4 th  prediction [0.00738185 0.95758872]
error in the 4 th iteration =  0.0018532081908864684
--------------------------------------
5 th  prediction [9.33944185e-04 9.77486832e-01]
error in the 5 th iteration =  0.0005077149896293903
--------------------------------------
6 th  prediction [1.17302387e-04 9.88164318e-01]
error in the 6 th iteration =  0.00014009713744961899
--------------------------------------
